In [1]:
#%%writefile get_proxy.py
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv


def get_proxy(which='free',now=False):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    ippos=[]
    with open(which_dict[which], 'r') as f:
            lines=f.readlines()
            for line in lines:
                line=line.strip()
                if len(line.split(":"))==2:
                    ippos.append(line)
    if now:
        sleep(1.5)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'a+') as f:
                        f.write(ippo+"\r\n")                
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    

In [ ]:
#kuai,ihuan,free
%time get_proxy('free')

In [2]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(30):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:ihuan_187.16.4.126:8080
OK:free_144.217.118.206:8080
1
OK:ihuan_51.158.123.250:8811
OK:free_186.42.124.130:65301
2
OK:free_117.58.241.164:52636
OK:ihuan_200.54.70.245:8080
3
OK:free_128.199.188.231:47503
OK:ihuan_94.205.254.82:3128
4
OK:ihuan_163.172.136.226:8811
OK:free_177.128.121.50:3128
5
OK:free_138.122.140.35:3128
OK:ihuan_39.137.95.72:80
6
OK:ihuan_175.42.128.2:9999
OK:free_83.228.74.251:32384
7
OK:ihuan_163.172.136.226:8811
OK:free_36.89.180.103:52092
8
OK:free_124.41.211.180:32861
OK:ihuan_109.183.189.238:53959
9
OK:ihuan_103.216.82.28:6666
OK:free_109.73.185.101:41258
10
OK:ihuan_123.200.20.6:8080
OK:free_222.124.154.19:23500
11
OK:free_103.242.14.8:48791
OK:ihuan_51.158.99.51:8811
12
OK:ihuan_123.200.20.6:8080
OK:free_104.41.26.153:8080
13
OK:ihuan_157.245.253.81:8080
OK:free_103.243.132.58:60356
14
OK:ihuan_163.172.152.52:8811
OK:free_60.248.199.206:80
15
OK:ihuan_117.197.42.21:46097
OK:free_75.151.213.85:8080
16
OK:free_118.174.232.234:44618
OK:ihuan_200.73.128.5:8080